## Joining Data in SQL

FROM DATAQUEST:

The most common way to join data using SQL is using an inner join. The syntax for an inner join is:

* Inner Join

An inner join works by including only rows from each table that have a match as specified using the ON clause. 

`SELECT [column_names] FROM [table_name_one]

INNER JOIN [table_name_two] ON [join_constraint];`

Example:
Let's use the facebook.db database which contains:
* facts table:

    * name -- the name of the country.
    * area -- the total land and sea area of the country.
    * population -- the population of the country.
    * birth_rate -- the birth rate of the country.
    * created_at -- the date the record was created.
    * updated_at -- the date the record was updated.


* Cities table

    * id - A unique ID for each city.
    * name - The name of the city.
    * population - The population of the city.
    * capital - Whether the city is a capital city: 1 if it is, 0 if it isn't.
    * facts_id - The ID of the country, from the facts table.


`SELECT * FROM facts
INNER JOIN cities ON cities.facts_id = facts.id
LIMIT 5;`

* Using alias:

`SELECT f.name, c.name FROM cities c
INNER JOIN facts f ON f.id = c.facts_id`


`SELECT * FROM facts
LEFT JOIN cities ON cities.facts_id = facts.id`

* Left Join


`SELECT COUNT(DISTINCT(name)) FROM facts;`

returns

`[["COUNT(DISTINCT(name))"], [261]]`


`COUNT(DISTINCT(facts_id)) FROM cities;`

returns

`[["COUNT(DISTINCT(facts_id))"], [210]]`

we can see that there are some countries in the facts table that don't have corresponding cities in the cities table (261 versus 210), which indicates we may have some incomplete data.

We can create a query to explore the missing data using a left join.

A left join includes all the rows that an inner join will select, plus any joins from the first (or left) table that don't have a match in the second table. 

`SELECT * FROM facts
LEFT JOIN cities ON cities.facts_id = facts.id`

We can use the returned null values for cities to filter our results to just the countries that don't exist in cities by adding the clause:

`SELECT f.name country, f.population FROM facts f
LEFT JOIN cities c ON c.facts_id = f.id where c.facts_id IS NULL` 

## GROUP BY SQL Statement

Let's consider the following fields in a given table recent_grads:

* Major_code - The major's code or ID
* Major - The name of the major
* Major_category - The broader category the major belongs to
* Total - The total number of people who studied the major
* Employed - The number of employed graduates

The GROUP BY SQL statement allows us to compute summary statistics by "group," or unique value. When we use this statement, SQL creates a group for each unique value in a column or set of columns (the same values we get when we use the DISTINCT statement), and then does the calculations for them.

Example:

`SELECT Major_category, AVG(Employed) / AVG(Total) AS share_employed 
FROM recent_grads 
GROUP BY Major_category`


## Having SQL Statement


When we want to filter on a column generated by a query, we can use the HAVING statement


Example:

`SELECT Major_category, AVG(Employed) / AVG(Total) AS share_employed 
FROM recent_grads 
GROUP BY Major_category 
HAVING share_employed > .8;`

## More Complex queries

Let's come back to facebook.db database.

We can write a query that generates output as shown above. The query includes the following columns, in order:
* country, the name of the country.
* urban_pop, the sum of the population in major urban areas belonging to that country.
* total_pop, the total population of the country.
* urban_pct, the percentage of the popularion within urban areas, calculated by dividing urban_pop by total_pop.
* Only countries that have an urban_pct greater than 0.5.
* Rows should be sorted by urban_pct in ascending order.


`SELECT  f.name country, SUM(c.population) urban_pop, f.population total_pop, SUM(c.population)/f.population urban_pct
FROM facts f INNER JOIN cities c
ON f.id == c.facts_id
GROUP BY country
HAVING urban_pct > 0.5
ORDER BY 4`


## Indexing Columns to Speed Up Queries

We can speed up `SELECT` queries filtering on columns by creating an index for these columns. 

* Return the query plan for a query that returns all rows where population is greater than 1000000 and where population_growth is less than 0.05.
* We're interested in all of the columns in the rows.
* Assign the query plan to query_plan_one and use print function to display the query plan.


In [1]:
import sqlite3
conn = sqlite3.connect("factbook.db")
q ='EXPLAIN QUERY PLAN SELECT * FROM facts WHERE population > 1000000 and population_growth < 0.05'
query_plan_one = conn.execute(q).fetchall()
print(query_plan_one)

[(0, 0, 0, 'SCAN TABLE facts')]


> You can see that the entire table is scanned. We can speed up the query creating an index.

* Create index
* Return the query plan for a query that returns all rows where population is greater than 1,000,000 and where population_growth is less than 0.05. Select only the population and population_growth columns.
* Assign the returned query plan to query_plan_four and use the print function to display it.


In [3]:
import sqlite3
conn = sqlite3.connect("factbook.db")
conn.execute("CREATE INDEX IF NOT EXISTS pop_pop_growth_idx ON facts(population, population_growth);")
q ='EXPLAIN QUERY PLAN SELECT population, population_growth FROM facts WHERE population > 1000000 AND population_growth < 0.05'
query_plan_four = conn.execute(q).fetchall()
print(query_plan_four)

[(0, 0, 0, 'SEARCH TABLE facts USING COVERING INDEX pop_pop_growth_idx (population>?)')]


* Return the query plan for a query that returns all rows where population is greater than 1,000,000. We're only interested in the population column.
* Assign the returned query plan to query_plan_five and use the print function to display it.

In [5]:
# import sqlite3
# conn = sqlite3.connect("factbook.db")
# We already created the index in the previous query
# conn.execute("CREATE INDEX IF NOT EXISTS pop_pop_growth_idx ON facts(population, population_growth);")
q ='EXPLAIN QUERY PLAN SELECT population FROM facts WHERE population > 1000000'
query_plan_five = conn.execute(q).fetchall()
print(query_plan_five)

[(0, 0, 0, 'SEARCH TABLE facts USING COVERING INDEX pop_pop_growth_idx (population>?)')]
